In [ ]:
import os
from net import Net
from data_generator import DataGenerator
from datetime import datetime
from tensorflow.compat.v1.data import Iterator
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

train_file = 'tables/train.txt'
val_file = 'tables/val.txt'

learning_rate = 0.0001
num_epochs = 10
batch_size = 128

dropout_rate = 0.5
num_classes = 2
train_layers = ['fc8', 'fc7', 'fc6']

display_step = 20

filewriter_path = "tmp/tensorboard"
checkpoint_path = "tmp/checkpoints"

if __name__ == '__main__':
    os.mkdir('tmp/')
    os.mkdir('tmp/checkpoints')

    with tf.device('/cpu:0'):
        tr_data = DataGenerator(train_file,
                                mode='training',
                                batch_size=batch_size,
                                num_classes=num_classes,
                                shuffle=True)
        val_data = DataGenerator(val_file,
                                 mode='inference',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=False)

        iterator = Iterator.from_structure(tr_data.data.output_types,
                                           tr_data.data.output_shapes)
        next_batch = iterator.get_next()

    training_init_op = iterator.make_initializer(tr_data.data)
    validation_init_op = iterator.make_initializer(val_data.data)

    x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
    y = tf.placeholder(tf.float32, [batch_size, num_classes])
    keep_prob = tf.placeholder(tf.float32)

    model = Net(x, keep_prob, num_classes, train_layers)

    score = model.fc8

    var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score, labels=y))

    with tf.name_scope("train"):
        gradients = tf.gradients(loss, var_list)
        gradients = list(zip(gradients, var_list))

        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.apply_gradients(grads_and_vars=gradients)

    for gradient, var in gradients:
        tf.summary.histogram(var.name + '/gradient', gradient)

    for var in var_list:
        tf.summary.histogram(var.name, var)

    tf.summary.scalar('cross_entropy', loss)

    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    tf.summary.scalar('accuracy', accuracy)

    merged_summary = tf.summary.merge_all()

    writer = tf.summary.FileWriter(filewriter_path)

    saver = tf.train.Saver()

    train_batches_per_epoch = int(np.floor(tr_data.data_size/batch_size))
    val_batches_per_epoch = int(np.floor(val_data.data_size / batch_size))

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        writer.add_graph(sess.graph)

        model.load_weights(sess)

        print(f'{datetime.now()} Start training...')
        print(f'{datetime.now()} Open Tensorboard at --logdir {filewriter_path}')

        for epoch in range(num_epochs):
            print(f'{datetime.now()} Epoch number: {epoch+1}')

            sess.run(training_init_op)
            for step in range(train_batches_per_epoch):

                img_batch, label_batch = sess.run(next_batch)
                sess.run(train_op, feed_dict={x: img_batch,
                                              y: label_batch,
                                              keep_prob: dropout_rate})
                if step % display_step == 0:
                    s = sess.run(merged_summary, feed_dict={x: img_batch, y: label_batch, keep_prob: 1.})
                    writer.add_summary(s, epoch*train_batches_per_epoch + step)

            print(f'{datetime.now()} Start validation')
            sess.run(validation_init_op)
            test_acc = 0.
            test_count = 0
            for _ in range(val_batches_per_epoch):
                img_batch, label_batch = sess.run(next_batch)
                acc = sess.run(accuracy, feed_dict={x: img_batch, y: label_batch, keep_prob: 1.})
                test_acc += acc
                test_count += 1
            test_acc /= test_count
            print(f'{datetime.now()} Validation Accuracy = {test_acc :.4f}')
            print(f'{datetime.now()} Saving checkpoint of model...')

            checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
            save_path = saver.save(sess, checkpoint_name)

            print(f'{datetime.now()} Model checkpoint saved at {checkpoint_name}')